In [1]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import warnings

In [2]:
# 경고 무시
warnings.filterwarnings('ignore')
# 페이지 접속
browser = webdriver.Chrome()
url = "https://www.yogiyo.co.kr/mobile/#/"
browser.maximize_window()
browser.get(url)
time.sleep(3)

location = '서울특별시 종로구 수송동 146-12 종로구청'
elem = browser.find_element_by_name("address_input")
elem.clear()
elem.send_keys(location)
time.sleep(1)
search_xpath = '''//*[@id="button_search_address"]/button[2]'''
search = browser.find_element_by_xpath(search_xpath)
browser.execute_script("arguments[0].click();", search)
time.sleep(2)

In [3]:
# 카테고리 선택
# 일단 치킨만!
cg = browser.find_element_by_xpath('/html/body/div[6]/div/div[1]/div[1]/div/div/div/ul/li[5]')
cg_title = cg.text
cg.click()

In [4]:
# 음식점 정렬
time.sleep(2)
browser.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select').click()
time.sleep(1)
browser.find_element_by_xpath('//*[@id="content"]/div/div[1]/div[2]/div/select/option[3]').click()

In [5]:
# 크롤링 함수
def review_crawling() :
    i = 1
    time.sleep(2)
    # store click
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]/a').click()
    ADDRESS = browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[6]/div[2]/p[3]/span').text # 가게 주소
    # 클린리뷰 tap
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(1)
    STORE = browser.find_element_by_xpath('/html/body/div[6]/div[2]/div[1]/div[1]/div[1]/span').text
    # 더보기 클릭
    cnt = 0
    while True :
        try :
            browser.find_element_by_class_name('btn-more').click()
            time.sleep(1.5)
            cnt += 1
            print("더보기 클릭 {}".format(cnt))
            # 더보기 개수제한
            #if cnt == 10:
                #break
        except:
            print("더보기 종료")
            break

    # 내용 가져오기
    df = pd.DataFrame(columns=[])
    STORES = []
    ID_list = []
    BIG_STARS = []
    DATES = []
    REVIEWS = []
    TOTAL_SCORES = []
    ADDRESSES = []
    req = browser.page_source
    soup = BeautifulSoup(req, 'html.parser')
    IDS = soup.select('span.review-id.ng-binding')
    BIG_STAR = str(soup.select(
        '#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div.restaurant-info > div.restaurant-content > ul > li:nth-child(1) > span')[
                       0].text)[-3:]
    how_many_r = 0
    for i in range(len(IDS)):
        try:
            how_many_r += 1
            total = len(IDS)
            ID_list.append(IDS[i].text)
            STAR_T = soup.select(
                '#review > li:nth-child({}) > div:nth-child(2) > div > span.category > span:nth-child(3)'.format(
                    i + 2))[0].text
            STAR_A = soup.select(
                '#review > li:nth-child({}) > div:nth-child(2) > div > span.category > span:nth-child(6)'.format(
                    i + 2))[0].text
            STAR_D = soup.select(
                '#review > li:nth-child({}) > div:nth-child(2) > div > span.category > span:nth-child(9)'.format(
                    i + 2))[0].text
            DATES.append(soup.select('span.review-time'.format(i + 2))[0].text)
            REVIEWS.append(soup.select('#review > li:nth-child({}) > p'.format(i + 2))[0].text)
            if (STAR_D != None) & (STAR_T != None) & (STAR_A != None):
                TOTAL_SCORE = round((float(STAR_T) + float(STAR_A) + float(STAR_D)) / 3.0, 1)
                TOTAL_SCORES.append(TOTAL_SCORE)
            else:
                TOTAL_SCORE = (float(STAR_T) + float(STAR_A)) / 2.0
                TOTAL_SCORES.append(TOTAL_SCORE)
            STORES.append(STORE)
            BIG_STARS.append(BIG_STAR)
            ADDRESSES.append(ADDRESS)
            print(f'총 리뷰{total}중 {how_many_r} 크롤링 완료!!!')
            # 크롤링 개수 제한
#             if how_many_r == 3:
#                 break
        except:
            continue

    df['업체명'] = STORES
    df['업체별'] = BIG_STARS
    df['주소'] = ADDRESS
    df['id'] = ID_list
    df['별점'] = TOTAL_SCORES
    df['작성일'] = DATES
    df['리뷰'] = REVIEWS
    print("크롤링 종료")
    # {STORE}로 csv 저장
    df.to_csv(f'{STORE}.csv', mode='a', header=False, encoding='utf-8-sig')
    print(df.head(3))
    print(f'{STORE}.csv 저장완료')
    browser.back()
    time.sleep(1)

In [ ]:
# 최소리뷰 개수 설정
# 설정한 수 이상의 리뷰만 크롤링
time.sleep(1)
i = 20
review_least = int(input("최소 리뷰 개수 설정 : "))
review_maximum = int(input("최대 리뷰 개수 설정 : "))
Q_how_many = int(input("몇개의 업체의 리뷰를 가져올까요?"))
how_many_s = 0
while True :
    count = browser.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div/table/tbody/tr/td[2]/div/div[2]/span[2]')
    review_count = int(count.text[3:])
    if review_count < review_least :
        break
    browser.find_element_by_xpath('//*[@id="content"]/div/div[5]/div/div/div['+str(i)+']/div').click()
    i = i+1
    if i>review_maximum:
        break
    review_crawling()
    how_many_s += 1
    if how_many_s == Q_how_many:
        break


print("크롤링을 완료하였습니다. 창을 닫습니다")
browser.close()

최소 리뷰 개수 설정 : 100
최대 리뷰 개수 설정 : 3000
몇개의 업체의 리뷰를 가져올까요?1
더보기 클릭 1
더보기 클릭 2
더보기 클릭 3
더보기 클릭 4
더보기 클릭 5
더보기 클릭 6
더보기 클릭 7
더보기 클릭 8
더보기 클릭 9
더보기 클릭 10
더보기 클릭 11
더보기 클릭 12
더보기 클릭 13
더보기 클릭 14
더보기 클릭 15
더보기 클릭 16
더보기 클릭 17
더보기 클릭 18
더보기 클릭 19
더보기 클릭 20
더보기 클릭 21
더보기 클릭 22
더보기 클릭 23
더보기 클릭 24
더보기 클릭 25
더보기 클릭 26
더보기 클릭 27
더보기 클릭 28
더보기 클릭 29
더보기 클릭 30
더보기 클릭 31
더보기 클릭 32
더보기 클릭 33
더보기 클릭 34
더보기 클릭 35
더보기 클릭 36
더보기 클릭 37
더보기 클릭 38
더보기 클릭 39
더보기 클릭 40
더보기 클릭 41
더보기 클릭 42
더보기 클릭 43
더보기 클릭 44
더보기 클릭 45
더보기 클릭 46
더보기 클릭 47
더보기 클릭 48
더보기 클릭 49
더보기 클릭 50
더보기 클릭 51
더보기 클릭 52
더보기 클릭 53
더보기 클릭 54
더보기 클릭 55
더보기 클릭 56
더보기 클릭 57
더보기 클릭 58
더보기 클릭 59
더보기 클릭 60
더보기 클릭 61
더보기 클릭 62
더보기 클릭 63
더보기 클릭 64
더보기 클릭 65
더보기 클릭 66
더보기 클릭 67
더보기 클릭 68
더보기 클릭 69
더보기 클릭 70
더보기 클릭 71
더보기 클릭 72
더보기 클릭 73
더보기 클릭 74
더보기 클릭 75
더보기 클릭 76
더보기 클릭 77
더보기 클릭 78
더보기 클릭 79
더보기 클릭 80
더보기 클릭 81
더보기 클릭 82
더보기 클릭 83
더보기 클릭 84
더보기 클릭 85
더보기 클릭 86
더보기 클릭 87
더보기 클릭 88
더보기 클릭 89
더보기 클릭 90
더보기 클릭 91
더보기 클릭 92
더보기 클릭 93
더보기 클릭 94
더보기 클릭 95
더보

총 리뷰2989중 213 크롤링 완료!!!
총 리뷰2989중 214 크롤링 완료!!!
총 리뷰2989중 215 크롤링 완료!!!
총 리뷰2989중 216 크롤링 완료!!!
총 리뷰2989중 217 크롤링 완료!!!
총 리뷰2989중 218 크롤링 완료!!!
총 리뷰2989중 219 크롤링 완료!!!
총 리뷰2989중 220 크롤링 완료!!!
총 리뷰2989중 221 크롤링 완료!!!
총 리뷰2989중 222 크롤링 완료!!!
총 리뷰2989중 223 크롤링 완료!!!
총 리뷰2989중 224 크롤링 완료!!!
총 리뷰2989중 225 크롤링 완료!!!
총 리뷰2989중 226 크롤링 완료!!!
총 리뷰2989중 227 크롤링 완료!!!
총 리뷰2989중 228 크롤링 완료!!!
총 리뷰2989중 229 크롤링 완료!!!
총 리뷰2989중 230 크롤링 완료!!!
총 리뷰2989중 231 크롤링 완료!!!
총 리뷰2989중 232 크롤링 완료!!!
총 리뷰2989중 233 크롤링 완료!!!
총 리뷰2989중 234 크롤링 완료!!!
총 리뷰2989중 235 크롤링 완료!!!
총 리뷰2989중 236 크롤링 완료!!!
총 리뷰2989중 237 크롤링 완료!!!
총 리뷰2989중 238 크롤링 완료!!!
총 리뷰2989중 239 크롤링 완료!!!
총 리뷰2989중 240 크롤링 완료!!!
총 리뷰2989중 241 크롤링 완료!!!
총 리뷰2989중 242 크롤링 완료!!!
총 리뷰2989중 243 크롤링 완료!!!
총 리뷰2989중 244 크롤링 완료!!!
총 리뷰2989중 245 크롤링 완료!!!
총 리뷰2989중 246 크롤링 완료!!!
총 리뷰2989중 247 크롤링 완료!!!
총 리뷰2989중 248 크롤링 완료!!!
총 리뷰2989중 249 크롤링 완료!!!
총 리뷰2989중 250 크롤링 완료!!!
총 리뷰2989중 251 크롤링 완료!!!
총 리뷰2989중 252 크롤링 완료!!!
총 리뷰2989중 253 크롤링 완료!!!
총 리뷰2989중 254 크롤